# Personio demo

## Installation
Install dlt using pip:

In [1]:
! pip install -q "dlt[duckdb]"
! dlt --version

dlt 0.3.25


Install additional dependencies for this demo:

In [15]:
!pip install -q pandas

## Initialize the verified source
This command will initialize the pipeline example with Personio as the source and `duckdb` as the destination:

In [2]:
! dlt --non-interactive init personio duckdb

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
No files to update, exiting


Set credentials in the `.dlt/secrets.toml` file

```toml
[sources.personio]
client_id = "client_id" # please set me up!
client_secret = "client_secret" # please set me up!
```

In [9]:
"""Pipeline to load personio data into Duckdb."""
import dlt
from personio import personio_source

pipeline = dlt.pipeline(
    pipeline_name="personio_1", destination='duckdb', dataset_name="personio_data"
)
resources = ["employees", "absences", "attendances"]
load_info = pipeline.run(
    personio_source().with_resources(*resources),
)
print(load_info)

Pipeline personio_1 completed in 37.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset personio_data
The duckdb destination used duckdb:////home/alenaastrakhantseva/dlthub/dlt_demos/personio_1.duckdb location to store data
Load package 1702550279.979736 is LOADED and contains no failed jobs


In [10]:
import duckdb
# a database 'chess_pipeline.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# this lets us query data without adding schema prefix to table names
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# list all tables
display(conn.sql("DESCRIBE"))

┌────────────┬──────────────────────┬──────────────────────┬──────────────────────┬────────────────────────┬───────────┐
│  database  │        schema        │         name         │     column_names     │      column_types      │ temporary │
│  varchar   │       varchar        │       varchar        │      varchar[]       │       varchar[]        │  boolean  │
├────────────┼──────────────────────┼──────────────────────┼──────────────────────┼────────────────────────┼───────────┤
│ personio_1 │ personio_data        │ _dlt_loads           │ [load_id, schema_n…  │ [VARCHAR, VARCHAR, B…  │ false     │
│ personio_1 │ personio_data        │ _dlt_pipeline_state  │ [version, engine_v…  │ [BIGINT, BIGINT, VAR…  │ false     │
│ personio_1 │ personio_data        │ _dlt_version         │ [version, engine_v…  │ [BIGINT, BIGINT, TIM…  │ false     │
│ personio_1 │ personio_data        │ absences             │ [id, status, comme…  │ [BIGINT, VARCHAR, VA…  │ false     │
│ personio_1 │ personio_data    

In [14]:
stats_table = conn.sql("SELECT * FROM absences").df()
display(stats_table)

,id,status,comment,start_date,end_date,days_count,half_day_start,half_day_end,time_off_type__type,time_off_type__attributes__id,...,employee__attributes__email__value,employee__attributes__email__type,employee__attributes__email__universal_id,created_by,certificate__status,created_at,updated_at,_dlt_load_id,_dlt_id,days_count__v_double
0,590798993,approved,,2020-06-04 00:00:00+02:00,2020-06-18 00:00:00+02:00,10.0,0,0,TimeOffType,2526184,...,christian.windholz@demo-sample.com,standard,email,None,not-required,2020-05-19 13:59:45+02:00,2022-11-09 10:34:43+01:00,1702550279.979736,ZOH6WcU/Mw79pw,NaN
1,590798995,approved,,2020-11-17 00:00:00+01:00,2020-11-23 00:00:00+01:00,5.0,0,0,TimeOffType,2526184,...,christian.windholz@demo-sample.com,standard,email,None,not-required,2020-05-19 13:59:45+02:00,2022-11-09 10:34:43+01:00,1702550279.979736,iVoElbavICrBQg,NaN
2,590799006,approved,,2020-07-13 00:00:00+02:00,2020-07-24 00:00:00+02:00,10.0,0,0,TimeOffType,2526184,...,greta.nordyke@demo-sample.com,standard,email,None,not-required,2020-05-19 13:59:47+02:00,2022-11-09 10:34:43+01:00,1702550279.979736,KwzGsFsSD8STMQ,NaN
3,590799008,approved,,2020-04-14 00:00:00+02:00,2020-04-24 00:00:00+02:00,9.0,0,0,TimeOffType,2526184,...,hans.huffer@demo-sample.com,standard,email,None,not-required,2020-05-19 13:59:47+02:00,2022-11-09 10:34:43+01:00,1702550279.979736,lp7p8mqjCtQdLA,NaN
4,590799054,approved,,2020-11-02 00:00:00+01:00,2020-11-06 00:00:00+01:00,5.0,0,0,TimeOffType,2526184,...,rene.storch@demo-sample.com,standard,email,None,not-required,2020-05-19 13:59:58+02:00,2022-11-09 10:34:43+01:00,1702550279.979736,zAARW7GcKi4soA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,590800087,approved,,2023-01-10 00:00:00+01:00,2023-01-10 00:00:00+01:00,NaN,1,0,TimeOffType,2526183,...,lars.keener@demo-sample.com,standard,email,None,not-required,2023-03-30 19:36:53+02:00,2023-03-30 19:36:53+02:00,1702550279.979736,H6f3xCST0dpb6A,0.5
1098,590800133,approved,,2023-05-19 00:00:00+02:00,2023-05-19 00:00:00+02:00,1.0,0,0,TimeOffType,2526183,...,lars.pressly@demo-sample.com,standard,email,None,not-required,2023-07-03 13:29:15+02:00,2023-07-03 13:29:15+02:00,1702550279.979736,lxUnne66li526g,NaN
1099,590800151,approved,,2023-08-07 00:00:00+02:00,2023-08-07 00:00:00+02:00,1.0,0,0,TimeOffType,2526183,...,rene.storch@demo-sample.com,standard,email,None,not-required,2023-09-26 14:57:51+02:00,2023-09-26 14:57:51+02:00,1702550279.979736,O3ul27ww0ZQj9A,NaN
1100,590800153,approved,,2023-07-11 00:00:00+02:00,2023-07-11 00:00:00+02:00,NaN,1,0,TimeOffType,2526183,...,lars.keener@demo-sample.com,standard,email,None,not-required,2023-09-26 14:57:51+02:00,2023-09-26 14:57:51+02:00,1702550279.979736,F0peF0qDF+Qw4Q,0.5
